In [4]:
import requests
import polars as pl

In [5]:
# [(Leaf, Seed)]
# herbs = [
#     ("Toadflax", "Toadflax seed"), ("Harralander", "Harralander seed"), ("Tarromin", "Tarromin seed"),
#     ("Marrentill", "Marrentill seed"), ("Ranarr weed", "Ranarr seed"),
#     ("Irit leaf", "Irit seed"), ("Avantoe", "Avantoe seed"), ("Kwuarm", "Kwuarm seed")
# ]

# https://oldschool.runescape.wiki/w/Item_IDs
details_url = "https://secure.runescape.com/m=itemdb_oldschool/api/catalogue/detail.json"

useful_keys = ["current", "today", "day30", "day90", "day180"]
herbs = [(2998, 5296), (255, 5294), (253, 5293),
         (251, 5292), (257, 5295),
         (259, 5297), (261, 5298), (263, 5299)]
results = {}

for leaf, seed in herbs:
    leaf_data, seed_data = None, None
    # Leaf
    leaf_response = requests.get(details_url, params={"item":leaf})
    if leaf_response.status_code == 200:
        leaf_data = leaf_response.json()["item"]
        print(leaf_data["name"])
    # Seed
    seed_response = requests.get(details_url, params={"item":seed})
    if seed_response.status_code == 200:
        seed_data = seed_response.json()["item"]
    if not leaf_data or not seed_data:
        print(f"Error with {leaf}:\n\t{leaf_response.status_code}\n\t{seed_response.status_code}")
    else:
        results[leaf_data["name"]] = (
            [leaf_data[key] for key in leaf_data.keys() if key in useful_keys],
            [seed_data[key] for key in seed_data.keys() if key in useful_keys]
        )
    # break

Toadflax
Harralander
Tarromin
Marrentill
Ranarr weed
Irit leaf
Avantoe
Kwuarm


In [6]:
results

{'Toadflax': ([{'trend': 'neutral', 'price': '3,767'},
   {'trend': 'negative', 'price': '- 1'},
   {'trend': 'positive', 'change': '+1.0%'},
   {'trend': 'positive', 'change': '+27.0%'},
   {'trend': 'positive', 'change': '+10.0%'}],
  [{'trend': 'neutral', 'price': 958},
   {'trend': 'positive', 'price': '+30'},
   {'trend': 'negative', 'change': '-9.0%'},
   {'trend': 'negative', 'change': '-4.0%'},
   {'trend': 'positive', 'change': '+3.0%'}]),
 'Harralander': ([{'trend': 'neutral', 'price': 728},
   {'trend': 'negative', 'price': '- 34'},
   {'trend': 'negative', 'change': '-46.0%'},
   {'trend': 'positive', 'change': '+10.0%'},
   {'trend': 'positive', 'change': '+10.0%'}],
  [{'trend': 'neutral', 'price': 34},
   {'trend': 'negative', 'price': '- 2'},
   {'trend': 'negative', 'change': '-29.0%'},
   {'trend': 'negative', 'change': '-8.0%'},
   {'trend': 'negative', 'change': '-5.0%'}]),
 'Tarromin': ([{'trend': 'neutral', 'price': 409},
   {'trend': 'neutral', 'price': 0},
   {'

In [28]:
# Prepare data for DataFrame with correct handling of 'today' field

def parse_int(val):
    if isinstance(val, str):
        val = val.replace(',', '').strip().lower()
        val = val.replace(' ', '').replace('+', '')
        if val.endswith('k'):
            return int(float(val[:-1]) * 1_000)
        elif val.endswith('m'):
            return int(float(val[:-1]) * 1_000_000)
        try:
            return int(val)
        except ValueError:
            return None
    elif isinstance(val, (int, float)):
        return int(val)
    return None

data = []
for herb, (leaf_stats, seed_stats) in results.items():
    row = {
        "herb": herb,
        "leaf_current": parse_int(leaf_stats[0]["price"]) if isinstance(leaf_stats[0], dict) and "price" in leaf_stats[0] else None,
        "seed_current": parse_int(seed_stats[0]["price"]) if isinstance(seed_stats[0], dict) and "price" in seed_stats[0] else None,
        "leaf_today": parse_int(leaf_stats[1]["price"]),  # 'today' is a direct value, not a dict
        "seed_today": parse_int(seed_stats[1]["price"]),  # 'today' is a direct value, not a dict
        "leaf_day30": leaf_stats[2]["change"] if isinstance(leaf_stats[2], dict) and "change" in leaf_stats[2] else leaf_stats[2],
        "seed_day30": seed_stats[2]["change"] if isinstance(seed_stats[2], dict) and "change" in seed_stats[2] else seed_stats[2],
        "leaf_day90": leaf_stats[3]["change"] if isinstance(leaf_stats[3], dict) and "change" in leaf_stats[3] else leaf_stats[3],
        "seed_day90": seed_stats[3]["change"] if isinstance(seed_stats[3], dict) and "change" in seed_stats[3] else seed_stats[3],
        "leaf_day180": leaf_stats[4]["change"] if isinstance(leaf_stats[4], dict) and "change" in leaf_stats[4] else leaf_stats[4],
        "seed_day180": seed_stats[4]["change"] if isinstance(seed_stats[4], dict) and "change" in seed_stats[4] else seed_stats[4],
    }
    data.append(row)

df = pl.DataFrame(data)
df

herb,leaf_current,seed_current,leaf_today,seed_today,leaf_day30,seed_day30,leaf_day90,seed_day90,leaf_day180,seed_day180
str,i64,i64,i64,i64,str,str,str,str,str,str
"""Toadflax""",3767,958,-1,30,"""+1.0%""","""-9.0%""","""+27.0%""","""-4.0%""","""+10.0%""","""+3.0%"""
"""Harralander""",728,34,-34,-2,"""-46.0%""","""-29.0%""","""+10.0%""","""-8.0%""","""+10.0%""","""-5.0%"""
"""Tarromin""",409,12,0,1,"""-64.0%""","""-14.0%""","""-9.0%""","""-33.0%""","""+17.0%""","""-25.0%"""
"""Marrentill""",597,20,-26,0,"""-44.0%""","""+42.0%""","""+54.0%""","""0.0%""","""+86.0%""","""+42.0%"""
"""Ranarr weed""",6984,28800,-53,-867,"""+3.0%""","""+7.0%""","""+16.0%""","""+5.0%""","""+13.0%""","""+31.0%"""
"""Irit leaf""",2819,77,-18,-2,"""+33.0%""","""-4.0%""","""+133.0%""","""-23.0%""","""+115.0%""","""-16.0%"""
"""Avantoe""",3746,746,60,21,"""-2.0%""","""-16.0%""","""+46.0%""","""-8.0%""","""+6.0%""","""-68.0%"""
"""Kwuarm""",3813,2393,-117,75,"""-2.0%""","""-15.0%""","""+32.0%""","""+44.0%""","""+4.0%""","""+46.0%"""


In [29]:
# Calculate new columns as described

df = df.with_columns([
    (df["leaf_current"] * 8.56 - df["seed_current"]).alias("leaf8.56_minus_seed"),
    (((df["leaf_current"] + df["leaf_today"]) * 8.56 - (df["seed_current"] + df["seed_today"]))
        .alias("leaf8.56_minus_seed_today"))
])
df

herb,leaf_current,seed_current,leaf_today,seed_today,leaf_day30,seed_day30,leaf_day90,seed_day90,leaf_day180,seed_day180,leaf8.56_minus_seed,leaf8.56_minus_seed_today
str,i64,i64,i64,i64,str,str,str,str,str,str,f64,f64
"""Toadflax""",3767,958,-1,30,"""+1.0%""","""-9.0%""","""+27.0%""","""-4.0%""","""+10.0%""","""+3.0%""",31287.52,31248.96
"""Harralander""",728,34,-34,-2,"""-46.0%""","""-29.0%""","""+10.0%""","""-8.0%""","""+10.0%""","""-5.0%""",6197.68,5908.64
"""Tarromin""",409,12,0,1,"""-64.0%""","""-14.0%""","""-9.0%""","""-33.0%""","""+17.0%""","""-25.0%""",3489.04,3488.04
"""Marrentill""",597,20,-26,0,"""-44.0%""","""+42.0%""","""+54.0%""","""0.0%""","""+86.0%""","""+42.0%""",5090.32,4867.76
"""Ranarr weed""",6984,28800,-53,-867,"""+3.0%""","""+7.0%""","""+16.0%""","""+5.0%""","""+13.0%""","""+31.0%""",30983.04,31396.36
"""Irit leaf""",2819,77,-18,-2,"""+33.0%""","""-4.0%""","""+133.0%""","""-23.0%""","""+115.0%""","""-16.0%""",24053.64,23901.56
"""Avantoe""",3746,746,60,21,"""-2.0%""","""-16.0%""","""+46.0%""","""-8.0%""","""+6.0%""","""-68.0%""",31319.76,31812.36
"""Kwuarm""",3813,2393,-117,75,"""-2.0%""","""-15.0%""","""+32.0%""","""+44.0%""","""+4.0%""","""+46.0%""",30246.28,29169.76


In [30]:
# Select and sort the relevant columns
df.select([
    "herb",
    "leaf8.56_minus_seed",
    "leaf8.56_minus_seed_today"
]).sort([
    "leaf8.56_minus_seed",
    "leaf8.56_minus_seed_today"
], descending=True)


herb,leaf8.56_minus_seed,leaf8.56_minus_seed_today
str,f64,f64
"""Avantoe""",31319.76,31812.36
"""Toadflax""",31287.52,31248.96
"""Ranarr weed""",30983.04,31396.36
"""Kwuarm""",30246.28,29169.76
"""Irit leaf""",24053.64,23901.56
"""Harralander""",6197.68,5908.64
"""Marrentill""",5090.32,4867.76
"""Tarromin""",3489.04,3488.04


In [ ]:
# Display the difference between leaf8.56_minus_seed_today and the next row's leaf8.56_minus_seed
df = df.with_columns([
    (df["leaf8.56_minus_seed_today"] - df["leaf8.56_minus_seed"].shift(-1)).alias("today_minus_next_seed")
])
df.select([
    "herb",
    "leaf8.56_minus_seed",
    "leaf8.56_minus_seed_today",
    "today_minus_next_seed"
])

herb,leaf8.56_minus_seed,leaf8.56_minus_seed_today,today_minus_next_seed
str,f64,f64,f64
"""Toadflax""",31287.52,31248.96,25051.28
"""Harralander""",6197.68,5908.64,2419.6
"""Tarromin""",3489.04,3488.04,-1602.28
"""Marrentill""",5090.32,4867.76,-26115.28
"""Ranarr weed""",30983.04,31396.36,7342.72
"""Irit leaf""",24053.64,23901.56,-7418.2
"""Avantoe""",31319.76,31812.36,1566.08
"""Kwuarm""",30246.28,29169.76,null


: 